In [3]:
import os

In [4]:
%pwd

'd:\\Programming\\Data Science\\NLP\\Text-Summarizer\\research'

In [5]:
os.chdir("../")

In [6]:
%pwd

'd:\\Programming\\Data Science\\NLP\\Text-Summarizer'

In [7]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [23]:

from src.textSummarizer.constants import *
from src.textSummarizer.utils.common import read_yaml, create_directories

In [24]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

In [25]:
import os
import urllib.request as request
import zipfile
from src.textSummarizer.logging import logger
from src.textSummarizer.utils.common import get_size

In [14]:
# !pip show python-box 

In [15]:
# !pip install python-box

  Obtaining dependency information for python-box from https://files.pythonhosted.org/packages/a8/0c/5c4a9aa6216265059fa3b2551274bb0cf21089cab5057c3f43be994ff75c/python_box-7.1.1-cp311-cp311-win_amd64.whl.metadata
   ---------------------------------------- 0.0/1.2 MB ? eta -:--:--
   ---- ----------------------------------- 0.1/1.2 MB 8.3 MB/s eta 0:00:01
   -------- ------------------------------- 0.3/1.2 MB 3.9 MB/s eta 0:00:01
   ------------- -------------------------- 0.4/1.2 MB 4.2 MB/s eta 0:00:01
   ------------------- -------------------- 0.6/1.2 MB 3.7 MB/s eta 0:00:01
   ------------------------ --------------- 0.7/1.2 MB 3.6 MB/s eta 0:00:01
   ------------------------------ --------- 0.9/1.2 MB 3.6 MB/s eta 0:00:01
   ----------------------------------- ---- 1.1/1.2 MB 3.6 MB/s eta 0:00:01
   ---------------------------------------- 1.2/1.2 MB 3.6 MB/s eta 0:00:00



[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [17]:
# !pip show PyYAML

In [18]:
# !pip install PyYAML

  Obtaining dependency information for PyYAML from https://files.pythonhosted.org/packages/b3/34/65bb4b2d7908044963ebf614fe0fdb080773fc7030d7e39c8d3eddcd4257/PyYAML-6.0.1-cp311-cp311-win_amd64.whl.metadata
   ---------------------------------------- 0.0/144.7 kB ? eta -:--:--
   ---------------------- ----------------- 81.9/144.7 kB 2.3 MB/s eta 0:00:01
   ---------------------------------------  143.4/144.7 kB 2.1 MB/s eta 0:00:01
   ---------------------------------------- 144.7/144.7 kB 1.7 MB/s eta 0:00:00



[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [21]:
# !pip show ensure

In [22]:
# !pip install ensure


[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [26]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")  

        
    
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [31]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2023-11-07 05:11:47,400: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-11-07 05:11:47,402: INFO: common: yaml file: params.yaml loaded successfully]
[2023-11-07 05:11:47,405: INFO: common: created directory at: artifacts]
[2023-11-07 05:11:47,407: INFO: common: created directory at: artifacts/data_ingestion]
[2023-11-07 05:11:50,569: INFO: 1434958058: artifacts/data_ingestion/data.zip download! with following info: 
Connection: close
Content-Length: 7903594
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "dbc016a060da18070593b83afff580c9b300f0b6ea4147a7988433e04df246ca"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 29BA:353AB:109624:1DF6A5:6549414A
Accept-Ranges: bytes
Date: Mon, 06 Nov 2023 23:41:47 GMT
Via: 1.1 varnish
X-Served-By: cache-hyd1100028-HYD
X-Cache: 